# IceNet CLI Usage

## Context

### Purpose
The IceNet library provides the ability to download, process, train and predict from end to end via a set of command-line interfaces. 

This notebook illustrates the CLI utilities that are available natively from the library and in conjunction with helpers scripts from the pipeline repository, for testing and producing operational forecasts.

### Modelling approach
This modelling approach allows users to immediately utilise the library for producing sea ice concentraion forecasts.

### Highlights
The key features of an end to end run are: 
* [Setup](#Setup)
* [Download](#Download) 
* [Process](#Process)
* [Train](#Train)
* [Predict](#Predict)

### Contributions
#### Notebook
James Byrne (author)

__Please raise issues [in this repository](https://github.com/icenet-ai/icenet-notebooks/issues) to suggest updates to this notebook!__ 

Contact me at _jambyr \<at\> bas.ac.uk_ for anything else...

#### Modelling codebase
James Byrne (code author), Tom Andersson (science author)

#### Modelling publications
Andersson, T.R., Hosking, J.S., Pérez-Ortiz, M. et al. Seasonal Arctic sea ice forecasting with probabilistic deep learning. Nat Commun 12, 5124 (2021). https://doi.org/10.1038/s41467-021-25257-4

#### Involved organisations
The Alan Turing Institute and British Antarctic Survey

## Setup

### Prerequisites

In order to undertake the following, I'm assuming you have a the following at your disposal:

* A host to run this on
* A working conda installation on that host
* Either a slurm cluster to submit jobs to or run locally
* Wherever you run, you want GPUs for training (predictions run fine without)
* Git, python and shell knowledge to a basic degree :-)
* There are numerous external facilities that we interface with, which it's assumed you're set up to use (otherwise check the options as they can be disabled/overlooked)
  * Data sources under [Climate and Sea Ice Data](#Climate-and-Sea-Ice-Data)
  * Wandb (Weights and Biases) - can be disabled when using `icenet_train`
  * Azure - we demonstrate native uploading which can be skipped if required

We'll assume that you're running in a local clone of `icenet-notebooks` for the sake of this tutorial, and __that one directory up we can deposit other repositories and folders__. If you already have some previous IceNet data available (as we do in `../data`) then you can symlink to it using `ln -s ../data`. The reason for this is described further below, as is the creation of this folder if it doesn't exist.

If you're starting right from scratch: 

```bash
git clone https://www.github.com/icenet-ai/icenet-pipeline.git ../notebook-pipeline
cd ../notebook-pipeline
```

### Environment Configuration

The icenet conda environment will need installing. To do this, [please follow the instructions for installing environments in icenet-pipeline](https://www.github.com/icenet-ai/icenet-pipeline) and enable the environment with `conda activate icenet`.

#### Commands

Once the icenet library is installed, you'll be able to access all commands made available by the library. Some are utilities that won't be covered, but using `icenet_<TAB>`-complete you should be able to see a list that includes (but ___is not limited to___):

* icenet_data_cmip
* icenet_data_era5
* icenet_data_hres
* icenet_data_masks
* icenet_data_sic
* icenet_dataset_create
* icenet_output
* icenet_predict
* icenet_process_cmip
* icenet_process_era5
* icenet_process_hres
* icenet_process_metadata
* icenet_process_sic
* icenet_train
* icenet_upload_azure

All of these commands are either directly or indirectly (through pipeline shell scripts) used in this notebook...

All commands accept options such as `-v` for turning on verbose logging and `-h` for obtaining help about what options they offer. ___As is best practice for all commands in *nix land, use `-h` to obtain information about options___.

### The idea behind end to end runs

The IceNet package is designed to support automated runs from end to end by exposing the above CLI operations. These are simple wrappers around the library itself, and __any__ step of this can be undertaken manually or programmatically by inspecting the relevant endpoints. 

___TL;DR: for those of you just wanting to skip straight to an end to end example in shell, [please look at the daily execution script](#Daily-execution)...___

The end to end execution methodology is illustrated by this diagram:

![Full IceNet operational workflow...](https://raw.githubusercontent.com/wiki/alan-turing-institute/IceNet-Project/Pipeline%20Layout.png)

The portion of this you're really interested in understand is in the green box however, with the IceNet-Pipeline directory (e.g. `green`) corresponding to the green box and thus being, essentially, an ephemeral environment. 

#### A tip behind source data

You'll see that `Source Data Store` is located outside the green box. Because of the expense and time required to interface with external sources, _we recommend the following step so that source data can be shared between environments_...

```bash
# Make a source data store outside our ephemeral environment
# For the sake of brevity the rest of the notebooks use a fresh environment
mkdir ../data
ln -s ../data
```

### Pipeline versus CLI verses Library usage

Though this notebook is tailored around use of the [IceNet-Pipeline repository](https://github.com/antarctica/IceNet-Pipeline) there is no dependency on this repository for using the `icenet_*` commands. The pipeline repository just offers helpers scripts written in [`bash`](https://tldp.org/LDP/abs/html) for running an end to end pipeline out of the box. 

You are welcome to use any arbitrary directory to run the CLI scripts below. However, when it comes to the sections on [training](#Train) and [prediction](#Predict), as well as [running daily predictions](#Daily execution), you'll notice that we leverage scripts from the pipeline repository. This is because these scripts interact with the [model ensembling tool from BAS](https://github.com/JimCircadian/model-ensembler) to train and predict across multiple models instances. 

The rule of thumb to follow: 

* Use the pipeline repository if you want to run the end to end IceNet processing out of the box.
* Adapt or customise this process using `icenet_*` commands described in this notebook and in the scripts contained in the pipeline repo.
* For ultimate customisation, you can interact with the IceNet repository programmatically (which is how the CLI commands operate.) For more information look at the [IceNet CLI implementations](https://github.com/JimCircadian/icenet2/blob/main/setup.py#L32) and the [library notebook](03.library_usage.ipynb), along with the [library documentation](#TODO). 

## Download

### Mask data

IceNet relies on some generated masks for training/prediction, which can be automatically generated very easily using `icenet_data_masks {north,south}`. Once performed, this does not need to be rerun under the pipeline directory...

In [1]:
!icenet_data_masks south

[23-11-22 09:51:22 :INFO    ] - Creating path: ./data/masks
--2022-11-23 09:51:22--  ftp://osisaf.met.no/reprocessed/ice/conc/v2p0/2000/01/ice_conc_sh_ease2-250_cdr-v2p0_200001021200.nc
           => ‘./data/masks/south/siconca/2000/01/.listing’
Resolving osisaf.met.no (osisaf.met.no)... 157.249.75.10
Connecting to osisaf.met.no (osisaf.met.no)|157.249.75.10|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /reprocessed/ice/conc/v2p0/2000/01 ... done.
==> PASV ... done.    ==> LIST ... done.

    [ <=>                                   ] 6,567       --.-K/s   in 0.03s   

2022-11-23 09:51:23 (217 KB/s) - ‘./data/masks/south/siconca/2000/01/.listing’ saved [6567]

--2022-11-23 09:51:23--  ftp://osisaf.met.no/reprocessed/ice/conc/v2p0/2000/01/ice_conc_sh_ease2-250_cdr-v2p0_200001021200.nc
           => ‘./data/masks/south/siconca/2000/01/ice_conc_sh_ease2-250_cdr-v2p0_200001021200.nc’
==> CWD not required.
=

### Climate and Sea Ice Data

Obtaining and preparing data is simply achieved using `icenet_data_*` commands (you need to __configure the [CDS API](https://cds.climate.copernicus.eu/) token yourself__), which share common arguments `hemisphere`, `start_date` and `end_date`. There are also implementation specific options worth reviewing under `--help`. We specify the variables and levels via these commands.

_Please ignore "NOT IMPLEMENTED YET", this is indicative of the commands not checking before overwriting files._

__The `-d` flag prevents the downloaded data from being downloaded each time.__

___Even small data ranges like this can take a while to retrieve (each variable in this case, for four months, is 3GB.) Please refer to [CDS requests page](https://cds.climate.copernicus.eu/cdsapp#!/yourrequests) to monitor ERA5 downloads...___

In [1]:
!icenet_data_era5 south -d --vars uas,vas,tas,zg --levels ',,,500|250' 2020-1-1 2020-4-30

[23-11-22 12:42:08 :INFO    ] - ERA5 Data Downloading
[23-11-22 12:42:08 :INFO    ] - Building request(s), downloading and daily averaging from ERA5 API
[23-11-22 12:42:08 :INFO    ] - Processing single download for uas @ None with 121 dates
[23-11-22 12:42:08 :INFO    ] - Processing single download for vas @ None with 121 dates
[23-11-22 12:42:08 :WARNING ] - NOT IMPLEMENTED YET, WE'LL JUST DOWNLOAD ANYWAY
[23-11-22 12:42:08 :INFO    ] - Processing single download for tas @ None with 121 dates
[23-11-22 12:42:08 :WARNING ] - NOT IMPLEMENTED YET, WE'LL JUST DOWNLOAD ANYWAY
[23-11-22 12:42:08 :INFO    ] - Processing single download for zg @ 500 with 121 dates
[23-11-22 12:42:08 :INFO    ] - Skipping actual download
[23-11-22 12:42:08 :INFO    ] - Processing single download for zg @ 250 with 121 dates
[23-11-22 12:42:08 :WARNING ] - NOT IMPLEMENTED YET, WE'LL JUST DOWNLOAD ANYWAY
[23-11-22 12:42:08 :INFO    ] - Skipping actual download
[23-11-22 12:42:08 :WARNING ] - NOT IMPLEMENTED YET,

In [2]:
!icenet_data_sic south -d 2020-1-1 2020-4-30

[23-11-22 12:44:45 :INFO    ] - OSASIF-SIC Data Downloading
[23-11-22 12:44:45 :INFO    ] - Creating path: ./data/osisaf
[23-11-22 12:44:45 :INFO    ] - Downloading SIC datafiles to .temp intermediates...
/data/hpcdata/users/jambyr/miniconda3/envs/icenet-notebooks/lib/python3.8/site-packages/icenet/data/sic/osisaf.py:269: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if el in self._invalid_dates:
[23-11-22 12:44:45 :INFO    ] - FTP opening
/data/hpcdata/users/jambyr/miniconda3/envs/icenet-notebooks/lib/python3.8/site-packages/flox/aggregations.py:259: RuntimeWarning: invalid value encountered in divide
  finalize=lambda sum_, count: sum_ / count,
/data/hpcdata/users/jambyr/miniconda3/envs/icenet-notebooks/lib/python3.8/site-packages/flox/aggregations.py:259: RuntimeWarning: invalid value 

By default, the IceNet commands regrid and rotates data as required to align with the OSISAF SIC data, which is used as the output for the dataset. Programmatic usage allows you to avoid this ([see notebook 03](0.3.library_usage)).

At time of writing there are the following downloaders: 

* `icenet_data_era5` - downloads [ERA5 reanalysis](https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset&keywords=((%20%22Product%20type:%20Reanalysis%22%20) data using either the CDS Toolbox or direct API
* `icenet_data_cmip` - downloads the prescribed experiments from [CMIP6](https://esgf-node.llnl.gov/search/cmip6/) for the original IceNet paper runs
* `icenet_data_hres` - downloads up to date [forecast generated data from the ECMWF MARS API](https://www.ecmwf.int/en/forecasts/datasets/catalogue-ecmwf-real-time-products)
* `icenet_data_sic` - downloads [OSISAF sea-ice concentration (SIC) data](https://osisaf-hl.met.no/v2p1-sea-ice-index)

## Process

Processing takes the data made available through the source data store and undertakes the necessary normalisation for use as input channels to the UNet architecture. This intermediary step means that the original source data can be reused numerous times with varying training, validation and test date setups.

### Command example

In [23]:
!icenet_process_era5 notebook_data south \
    -ns 2020-1-1 -ne 2020-3-31 -vs 2020-4-3 -ve 2020-4-23 -ts 2020-4-1 -te 2020-4-2 \
    -l 1 --abs uas,vas --anom tas,zg500,zg250

!icenet_process_sic notebook_data south \
    -ns 2020-1-1 -ne 2020-3-31 -vs 2020-4-1 -ve 2020-4-20 -ts 2020-4-1 -te 2020-4-2 \
    -l 1 --abs siconca

!icenet_process_metadata notebook_data south

2022-12-05 17:35:23.690232: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 17:35:37.160044: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/data/hpcdata/users/jambyr/miniconda3/envs/icenet-notebooks/lib/
2022-12-05 17:35:37.160394: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/data/hpcdata/users/jambyr/miniconda3/envs/icenet-noteb

Consulting the command options will make the above more obvious (as well as further options) but a few things we can note that are helpful: 

* Options `-ns`, `-ne`, `-vs`, `-ve`, `-ts`, `-te`, which correspond to training, validation and test sets, allow ranges to be comma-delimited. The above example produces a split training set, for example, that spans two periods: 2000-2009 and 2011-2019.
* These date ranges can be randomised and subsampled using `-d`, __though this is still a bit experimental__
* The `-l` option (which is for `--lag`) specified the number of days back we look at input data variables for the output in question.

There are plenty of other options available for preprocessing the data, but it should be noted that whilst this is not strongly coupled to dataset creation, options like the lag specified here might influence the creation of datasets in the next step. 

These commands, especially with decadal ranges, can take a long time (12+ hours) to complete depending on the hosts/storage in use.

### Dataset creation

Once the above preprocessing is taken care of datasets can easily be created thus. This operation _creates a cached dataset_ in the filesystem that can be fed in for training runs. 

In [24]:
!icenet_dataset_create -l 1 -fd 7 -ob 2 -w 4 notebook_data south

2022-12-05 17:39:53.518040: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 17:39:55.711541: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/data/hpcdata/users/jambyr/miniconda3/envs/icenet-notebooks/lib/
2022-12-05 17:39:55.711684: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/data/hpcdata/users/jambyr/miniconda3/envs/icenet-noteb

The common options used here: 

* `-fn` allows us to specify how far forward to forecast to. For this example we're limiting to 7 days based on the limited amount of SIC groud truth data we downloaded.
* `-l` as in the preprocessing stage. If experimenting and using full date ranges, creating a dataset with a different lag can save having to reprocess everything.
* `-ob` is the output batch size for the tfrecords. It is advisable to keep this smaller except where there are seriously large numbers of sets, preferably near to the expected size being used for training.
* `-w` specifies the number of worker subprocesses to use for producing the output. Probably advisable to keep this below the number of cores on your host! :) 

#### Config-only operation / Prediction datasets

Datasets used to predict don't benefit from caching, so adding the `-c` option and dropping `-w` and `-ob` will create a configuration for the dataset without writing sets to disk. You can also use this option to create a dataset that is fed directly from the preprocessed data, though bear in mind, depending on your infrastructure, that this requires the batches to be created on the fly and can have a significant impact on performance. By specifying `-fn` we ensure the dataset is given a different name to the previously cached one above (though this is more commonly used for prediction datasets where caching isn't necessary...) 

In [25]:
!icenet_dataset_create -fd 7 -l 1 -c -fn notebook_raw_dataset notebook_data south

2022-12-05 17:41:57.988652: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 17:41:58.997186: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/data/hpcdata/users/jambyr/miniconda3/envs/icenet-notebooks/lib/
2022-12-05 17:41:58.997353: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/data/hpcdata/users/jambyr/miniconda3/envs/icenet-noteb

## Train

Once the dataset is prepared, running a network is then as simple as using `icenet_train` with the appropriate parameters. Some key parameters are illustrated in the following commands:
 

In [26]:
!icenet_train notebook_data notebook_testrun 42 -b 2 -e 10 -m -qs 4 -w 4 -n 0.6

2022-12-05 17:42:09.429760: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 17:42:10.164723: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/data/hpcdata/users/jambyr/miniconda3/envs/icenet-notebooks/lib/
2022-12-05 17:42:10.164883: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/data/hpcdata/users/jambyr/miniconda3/envs/icenet-noteb

In [27]:
!icenet_train notebook_data notebook_testrun 42 -b 2 -e 2 -m -qs 4 -w 4 -n 0.6 \
    -p ./results/networks/notebook_testrun/notebook_testrun.network_notebook_data.42.h5 

2022-12-05 17:46:00.290827: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 17:46:01.249023: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/data/hpcdata/users/jambyr/miniconda3/envs/icenet-notebooks/lib/
2022-12-05 17:46:01.249181: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/data/hpcdata/users/jambyr/miniconda3/envs/icenet-noteb

These runs demonstrate using the aforementioned dataset, in `-b` batches of 4 for a run of `-e` five epochs. Using `-m` for multiprocessing we enable up to `-w` four process workers to load data at a time into a data queue `-qs` of length four. We could specify a `-r` ratio we use only 0.2x of the files from the dataset (_useful when testing on a low power machine with a large dataset, but unnecessary with our example here_) supplying a UNet built with 0.6x the `-n` numbers of filters as normal. 

With the second command we `-p` pickup the output weights from the previous run to continue training.

There are a few things to note about the `icenet_train` and `icenet_predict` (see [the prediction section below](#Predict)) commands and the switches they provide: 

* Common switches such as `-n` should be applied consistently between training and prediction. 
* These commands work with __individual network runs__ (see the next section).

### Ensemble running

For producing forecasts in the described pipeline we actually run a set of models using the [model-ensembler](https://github.com/JimCircadian/model-ensembler) tool and as such there are convenience scripts for doing this as part of the end to end run. 

To do this we need to be running in a clone of the `icenet-pipeline` repository.

In [28]:
!./run_train_ensemble.sh \
    -b 4 -e 5 -f 0.6 -p bashpc.sh -m 64gb -q 4 -j 5 \
    notebook_data notebook_data notebook_ensemble

ARGS: -b 4 -e 5 -f 0.6 -p bashpc.sh -m 64gb -q 4 -j 5 notebook_data notebook_data notebook_ensemble
ARGS = -x arg_batch=4 arg_epochs=5 arg_filter_factor=0.6 arg_prep=bashpc.sh mem=64gb arg_queue=4 , Leftovers: notebook_data notebook_data notebook_ensemble
Running model_ensemble ./tmp.jwcpS27GTc.train slurm -x arg_batch=4 arg_epochs=5 arg_filter_factor=0.6 arg_prep=bashpc.sh mem=64gb arg_queue=4 
[05-12-22 17:49:01    :INFO    ] - Model Ensemble Runner
[05-12-22 17:49:01    :INFO    ] - Validated configuration file ./tmp.jwcpS27GTc.train successfully
[05-12-22 17:49:01    :INFO    ] - Importing model_ensembler.cluster.slurm
[05-12-22 17:49:01    :INFO    ] - Running batcher
[05-12-22 17:49:01    :INFO    ] - Running command: mkdir -p ./results/networks
[05-12-22 17:49:01    :INFO    ] - Start batch: 2022-12-05 17:49:01.487298
[05-12-22 17:49:01    :INFO    ] - Running cycle 1
[05-12-22 17:49:01    :INFO    ] - Start run notebook_ensemble-0 at 2022-12-05 17:49:01.498830
[05-12-22 17:49:0

Many of the arguments are equivalent to the above `icenet_train` command. However, the `-n` filters factor is actually `-f` in this example (note that because I'm running on a cluster I've doubled this) and we have additional arguments `-n` for the node to run on, `-p` for the pre_run script to use and `-j` for the number of simultaneous runs to execute on the SLURM cluster we use at BAS. However, these arguments are not necessarily required for other clusters, nor is the model-ensembler limited to running on SLURM (it can, at present, also run locally.)  

The pipeline repository shell scripts that provide this functionality are easily adaptable, as well as the ensemble itself which is stored in the pipeline repository under `/ensemble/`.

_Please review the `-h` help option for the script to gain further insight the options available._

## Predict

One the network is trained it is possible to run any suitable sets through the network for training. __This is the purpose of configuration only datasets__ which are used by the `run_predict_ensemble` to, similarly to the training process, run predictions through all of the ensemble members. 

To run an individual sets through the test network from the test dataset we produced earlier can be easily achieved. The steps are to create a date file, which can be produced from the configuration created by `icenet_process` in the [processing section](#Process). This date file then can be supplied to the `icenet_predict` command to produce files using either cached data (useful for test data prepared at the same time as the training and validation sets) or directly from the normalised data (as is the case for nearly all data that isn't part of the training run.)

In [29]:
!./loader_test_dates.sh notebook_data | tee testdates.csv

2020-04-01
2020-04-02


In [30]:
!icenet_predict -n 0.6 -t \
    notebook_data notebook_testrun example_south_forecast 42 testdates.csv

2022-12-05 17:57:35.484319: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 17:57:36.476585: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/data/hpcdata/users/jambyr/miniconda3/envs/icenet-notebooks/lib/
2022-12-05 17:57:36.476742: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/data/hpcdata/users/jambyr/miniconda3/envs/icenet-noteb

The example uses the cached test data from the training run, but the process is the same for any other processed data with only the need to _omit the `-t` option, which specifies to source from cached test data_.

### Outputs

In the above example, there are three outputs: 

* __forecast__: the ___predicted___ forecast data from the model output layer
* __outputs__: the outputs from the data loader which would be used for training
* __weights__: the generated sample weights from the data loader for the training sample

The outputs initially are stored as Numpy arrays under the `results` directory thusly: 

```
results/predict/example_south_forecast/notebook_testrun.42/2020_04_01.npy
results/predict/example_south_forecast/notebook_testrun.42/2020_04_02.npy
```

With associated inputs, output and weights stored within subfolders.

### Ensemble running

When producing daily forecasts for IceNet we train on an ensemble of models and also run predictions across them producing a mean and error across that model ensemble. To do this the pipeline repository offers the `run_predict_ensemble` which operates similarly to the above training script. An example of running the ensemble: 

In [ ]:
!./run_predict_ensemble.sh \
    -b 1 -f 0.6 -p bashpc.sh \
    notebook_ensemble notebook_data example_south_ensemble_forecast testdates.csv

ARGS: -b 1 -f 0.6 -p bashpc.sh notebook_ensemble notebook_data example_south_ensemble_forecast testdates.csv
ARGS = -x arg_batch=1 arg_filter_factor=0.6 arg_prep=bashpc.sh , Leftovers: notebook_ensemble notebook_data example_south_ensemble_forecast testdates.csv
Running model_ensemble ./tmp.PwiZ36tOKl.predict slurm -x arg_batch=1 arg_filter_factor=0.6 arg_prep=bashpc.sh 
[05-12-22 19:42:56    :INFO    ] - Model Ensemble Runner
[05-12-22 19:42:56    :INFO    ] - Validated configuration file ./tmp.PwiZ36tOKl.predict successfully
[05-12-22 19:42:56    :INFO    ] - Importing model_ensembler.cluster.slurm
[05-12-22 19:42:56    :INFO    ] - Running batcher
[05-12-22 19:42:56    :INFO    ] - Start batch: 2022-12-05 19:42:56.062211
[05-12-22 19:42:56    :INFO    ] - Running cycle 1
[05-12-22 19:42:56    :INFO    ] - Running command: /usr/bin/ln -s ../../data
[05-12-22 19:42:56    :INFO    ] - Start run example_south_ensemble_forecast-0 at 2022-12-05 19:42:56.074011
[05-12-22 19:42:56    :INFO 

As with the previous example, the individual numpy outputs, samples and sample weights are deposited into `/results/predict` for each ensemble member. However, the ensemble also runs `icenet_output` to generate __a CF-compliant NetCDF containing the forecasts requested__ which can then be post-processed or [deposited to an external location](#Uploading-to-Azure) (which is the platform for the [wider IceNet forecasting infrastructure](https://github.com/alan-turing-institute/IceNet-Project). 

In [7]:
!icenet_output -o results/predict example_south_forecast notebook_data testdates.csv

2022-11-23 23:44:59.809415: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-23 23:45:00.593340: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/data/hpcdata/users/jambyr/miniconda3/envs/icenet-notebooks/lib/
2022-11-23 23:45:00.593489: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/data/hpcdata/users/jambyr/miniconda3/envs/icenet-noteb

Note that the ensemble run automatically handles this generation of output for the ensemble. For a single run this is relatively meaningless as there is only a single model making predictions, giving no uncertainty quantification, __so this is provided as an example only__. _Please review the `-h` help option for the script to gain further insight the options available._

### Uploading to Azure

The following command uploads a specific date from the `icenet_output` produced dataset to an Azure storage blob storage account.  

In [ ]:
!icenet_upload_azure results/predict/example_south_ensemble_forecast.nc 2020-04-30

## Other Pipeline Considerations

### A bit more information on ensemble runs

#### Cleaning up runs

Ensemble runs take place under `/ensemble/` in the pipeline folder and ARE NOT deleted after they've happened, to allow for debugging. Commonly, the ensemble configurations will contain a delete task to remove the extraneous run folders. __In the meantime this should be done manually__ after running `run_train_ensemble` or `run_predict_ensemble`.

The only exception to this is the use of `run_daily.sh` (see below) which does clean up prior to rerunning. 

### Daily execution

Daily execution is facilitated in the pipeline by using [`run_daily.sh`](https://github.com/antarctica/IceNet-Pipeline/blob/main/run_daily.sh). This wraps all the necessary steps to perform the following sequence for producing forecasts from yesterday for the next 93 days, for both northern and southern hemispheres. 

* Removes any old ensemble runs
* Downloads [HRES forecast data from the ECMWF MARS API](https://www.ecmwf.int/en/forecasts/datasets/catalogue-ecmwf-real-time-products)
* Processes the HRES and necessary training metadata to produce a data loader
* Creates a dataset configuration for it
* Runs a [prediction ensemble](#Predict) to produce a NetCDF
* Uploads to the necessary endpoint

#### Automation

With the above shell script it's trivial to automate using cron. Of course this is simply for demonstration, with more complex workflow managers offering far great flexibility especially when considering analysis of the produced forecasts.

```bash
# We assume your environment is configured appropriately to run conda from cron files, for example by adding...
#
# SHELL=/bin/bash
# BASH_ENV=~/.bashrc_env
#
# With conda initialisation in bashrc_env at the top of your crontab
25 9 * * * conda activate icenet; cd $HOME/hpc/icenet/pipeline && bash run_daily.sh >$HOME/daily.log 2>&1; conda deactivate
```

TODO: more information on the usage of this command.

## Summary

Within this notebook we've attempted to give a full crash course to running the CLI tools both __manually__ and using the __pipeline helper scripts__. This is the first of four (currently) notebooks contained within the pipeline repository, covering further information: 

* [Data structure and analysis](02.data_analysis.ipynb): understand the structure of the data stores and products created by these workflows and what tools currently exist in IceNet to looks over them.
* [Library usage](03.library_usage.ipynb): understand how to programmatically perform an end to end run.
* [Library extension](04.library_extension.ipynb): understand why and how to extend the IceNet library.

## Version
- IceNet Codebase: v0.2.0.dev7